<a href="https://colab.research.google.com/github/arva29/AMDproject/blob/main/UkraineWarTweetsProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# this is meant to be run on google colab

!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#!wget -q http://www-eu.apache.org/dist/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
!wget -q https://archive.apache.org/dist/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
!tar xf spark-2.4.5-bin-hadoop2.7.tgz
!pip install -q findspark

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"

import findspark
findspark.init("spark-2.4.5-bin-hadoop2.7") #SPARK_HOME
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

sc = spark.sparkContext

In [3]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  
# Then move kaggle.json into the folder where the API expects to find it.
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json
User uploaded file "kaggle.json" with length 70 bytes


In [4]:
!kaggle datasets download -d bwandowando/ukraine-russian-crisis-twitter-dataset-1-2-m-rows

100% 12.4G/12.4G [02:02<00:00, 103MB/s]
100% 12.4G/12.4G [02:02<00:00, 109MB/s]


In [5]:
!unzip ukraine-russian-crisis-twitter-dataset-1-2-m-rows.zip

Archive:  ukraine-russian-crisis-twitter-dataset-1-2-m-rows.zip
  inflating: 0819_UkraineCombinedTweetsDeduped.csv.gzip  
  inflating: 0820_UkraineCombinedTweetsDeduped.csv.gzip  
  inflating: 0821_UkraineCombinedTweetsDeduped.csv.gzip  
  inflating: 0822_UkraineCombinedTweetsDeduped.csv.gzip  
  inflating: 0823_UkraineCombinedTweetsDeduped.csv.gzip  
  inflating: 0824_UkraineCombinedTweetsDeduped.csv.gzip  
  inflating: 0825_UkraineCombinedTweetsDeduped.csv.gzip  
  inflating: 0826_UkraineCombinedTweetsDeduped.csv.gzip  
  inflating: 0827_UkraineCombinedTweetsDeduped.csv.gzip  
  inflating: 0828_UkraineCombinedTweetsDeduped.csv.gzip  
  inflating: 0829_UkraineCombinedTweetsDeduped.csv.gzip  
  inflating: 0830_UkraineCombinedTweetsDeduped.csv.gzip  
  inflating: 0831_UkraineCombinedTweetsDeduped.csv.gzip  
  inflating: 0901_UkraineCombinedTweetsDeduped.csv.gzip  
  inflating: 0902_UkraineCombinedTweetsDeduped.csv.gzip  
  inflating: 0903_UkraineCombinedTweetsDeduped.csv.gzip  
  inflat

I extract the collection of tweets from the (23 of September ?? ) and I will work on this.

In [6]:
import csv
import gzip

data_list = []

with gzip.open('1008_UkraineCombinedTweetsDeduped.csv.gzip', 'rt') as f:
    reader = csv.DictReader(f)
    for row in reader:
        data_list.append(row)

rdd = sc.parallelize(data_list)

In [8]:
rdd.count()

86471

In [9]:
rdd.first()

OrderedDict([('', '130404'),
             ('userid', '1603709857'),
             ('username', 'trajanpublisher'),
             ('acctdesc',
              "We are Trajan Media, owner of Canada's premier numismatic and philatelic publications –\xa0Canadian Coin News and Canadian Stamp News –\xa0plus Coin & Stamp Supplies."),
             ('location', 'Canada'),
             ('following', '190'),
             ('followers', '916'),
             ('totaltweets', '7215'),
             ('usercreatedts', '2013-07-18 15:23:57'),
             ('tweetid', '1578535646146199554'),
             ('tweetcreatedts', '2022-10-08 00:00:00'),
             ('retweetcount', '1'),
             ('text',
              'Canada joined 15 countries whose flags grace the obverse of Ukraine’s first commemorative coin issued since the beginning of Russia’s renewed invasion this February.\n\nRead more: https://t.co/vRhROceH9x\n\n#numismatics #Ukraine️ #collectcoins'),
             ('hashtags',
              "[{'text':

Tokenize tweet by removing puntuation and stop words in order to have only meaningful words to understand the meaning of the tweet. I also put all the words lower case and remove the link at the end of the tweets.

In [7]:
!pip install clean-text

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 175 kB 7.5 MB/s 
     |████████████████████████████████| 53 kB 2.0 MB/s 
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171046 sha256=09e840f8d893a59a272141061efbbf0b8825a695b77a7075f502b3df02c752dc
  Stored in directory: /root/.cache/pip/wheels/8a/4e/b6/57b01db010d17ef6ea9b40300af725ef3e210cb1acfb7ac8b6
Successfully built emoji


In [8]:
import nltk
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
from string import punctuation
from cleantext import clean

nltk.download('stopwords')
stopw = set(stopwords.words('english'))

tokenizer = TweetTokenizer()
#tokenize = lambda text: [word for word in tokenizer.tokenize(clean(text, no_urls=True, no_emails=True, no_emoji=True, no_line_breaks=True, replace_with_email="", replace_with_url="")) if word not in punctuation and word not in stopw]
tokenize = lambda text: [word for word in tokenizer.tokenize(clean(text, no_urls=True, no_emails=True, no_emoji=True, no_line_breaks=True, replace_with_email="", replace_with_url="")) if word not in punctuation]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [9]:
import numpy as np

def sort(list):
  list.sort()
  return list

en_tweets = rdd.filter(lambda x: x.get("language")=='en').map(lambda x: (x, x.get("text")))

tweets_tokenized = en_tweets.map(lambda x: (x[0].get("tweetid"), tokenize(x[1])))

tweets_tokenized.first()

('1578535646146199554',
 ['canada',
  'joined',
  '15',
  'countries',
  'whose',
  'flags',
  'grace',
  'the',
  'obverse',
  'of',
  "ukraine's",
  'first',
  'commemorative',
  'coin',
  'issued',
  'since',
  'the',
  'beginning',
  'of',
  "russia's",
  'renewed',
  'invasion',
  'this',
  'february',
  'read',
  'more',
  '#numismatics',
  '#ukraine',
  '#collectcoins'])

In [10]:
from pyspark.sql.types import StructType,StructField, StringType, ArrayType

dfSchema = StructType([       
    StructField('tweet_id', StringType(), True),
    StructField('tokens', ArrayType(StringType()), True)
])

tmp = tweets_tokenized.map(lambda x: (x[0], x[1]))
tweetsDF = spark.createDataFrame(tmp, dfSchema)

tweetsDF.show()

+-------------------+--------------------+
|           tweet_id|              tokens|
+-------------------+--------------------+
|1578535646146199554|[canada, joined, ...|
|1578535651984650240|[good, night, #kh...|
|1578535652248850432|[us, invests, 290...|
|1578535654149038080|[go, fuck, yourse...|
|1578535658691629056|[today, 26:30, wa...|
|1578535679713497089|[#saynomore, #usa...|
|1578535680585916417|[new, podcast, sb...|
|1578535686818304000|[another, deluded...|
|1578535704694779907|[liberation, is, ...|
|1578535714945658882|[perhaps, we, jus...|
|1578535720276590593|[thank, the, #usa...|
|1578535722335682560|[to, the, tune, o...|
|1578535723413950464|[the, world, does...|
|1578535726266060800|[the, next, 2, ye...|
|1578535753834934272|[zelensky, the, w...|
|1578535760596463616|[@lesiavasylenko,...|
|1578535802534907904|[#biden, finalize...|
|1578535866087022592|[grand, larceny, ...|
|1578535891491954688|[#weekendvibes, #...|
|1578535911444254720|[#odessa, #odesa,...|
+----------

## *K*-Shingles

In [11]:
K = int(input("Enter the number K for shingles dimension: "))

def build_shingles(tokens, K):
  shingles = list()
  tmp = ""

  for i in range(len(tokens)):
    tmp = tokens[i]
    if((len(tokens) - (i)) < K):
      break;
    for k in range(K - 1):
      tmp += " " + tokens[i + (k + 1)]
    shingles.append(tmp)

  return shingles

shingles_rdd = tweetsDF.rdd.map(lambda x: (x[0], sort(build_shingles(x[1], K))))
shingles_rdd.first()

Enter the number K for shingles dimension: 3


('1578535646146199554',
 ['#numismatics #ukraine #collectcoins',
  '15 countries whose',
  "beginning of russia's",
  'canada joined 15',
  'coin issued since',
  'commemorative coin issued',
  'countries whose flags',
  'february read more',
  'first commemorative coin',
  'flags grace the',
  'grace the obverse',
  'invasion this february',
  'issued since the',
  'joined 15 countries',
  'more #numismatics #ukraine',
  "obverse of ukraine's",
  "of russia's renewed",
  "of ukraine's first",
  'read more #numismatics',
  'renewed invasion this',
  "russia's renewed invasion",
  'since the beginning',
  'the beginning of',
  'the obverse of',
  'this february read',
  "ukraine's first commemorative",
  'whose flags grace'])

Cast from list to set and then to list again in order to remove all the duplicate elements

In [12]:
shingles_bag = sort(list(set(shingles_rdd.flatMap(lambda x: x[1]).collect())))
shingles_bag[0:10]

['##biggerthanme #osimhen #dazn',
 '##crimea #tma2022 #thefactmusicawards2022',
 '##crimea as #ukraine',
 '##dpr allied forces',
 '##droz #democraticviolence #gpexplorer',
 '##iranianlivesmatter #iranprotests2022 #tv3newday',
 '##kimjongun and #danielortega',
 '##kpworldtoursingapore #kerch #weverse',
 '##marvel #ironman #thor',
 '##nctdream #pakistanzindabad #case143']

I use integers from 0 to *n* (with n equal to the length of shgles_bag) to store the shingles list

In [42]:
shingles_bag_id = list(range(0, len(shingles_bag)))
len(shingles_bag_id)

572145

In [ ]:
def convertShingles(shingles, BoS):
  listToReturn = [0]*len(BoS)
  start_index = 0;

  for s in shingles:
    for i in range(start_index, len(BoS)):
      if(s == BoS[i]):
        listToReturn[i] = 1
        start_index = i
        break;

  return listToReturn

shingles_int_rdd = shingles_rdd.map(lambda x: (x[0], convertShingles(x[1], shingles_bag)))
shingles_int_rdd.first()

Hashing function for permutation

In [68]:
def generatePermutation(arrayToHash, a, b):
  arrayToReturn = arrayToHash.copy()
  
  for i in range(len(arrayToHash)):
    arrayToReturn[i] = (a * arrayToHash[i] + b) % len(arrayToHash)

  return arrayToReturn

n = int(input("Insert number n of permutation desired: "))
permutation_matrix = list()
#shingles_bag_id = list(range(0, 1000000))

for i in range(n):
  #Generate 2 random numbers in order to have different hashing by randomly chaging only these two numbers
  a = np.random.randint(1000) 
  b = np.random.randint(1000)
  #print(a, b)

  permutation_matrix.append(generatePermutation(shingles_bag_id, a, b))
  print(permutation_matrix[i][:30])

Insert number n of permutation desired: 10
[523888, 94963, 238183, 381403, 524623, 95698, 238918, 382138, 525358, 96433, 239653, 382873, 526093, 97168, 240388, 383608, 526828, 97903, 241123, 384343, 527563, 98638, 241858, 385078, 528298, 99373, 242593, 385813, 529033, 100108]
[129836, 494711, 287441, 80171, 445046, 237776, 30506, 395381, 188111, 552986, 345716, 138446, 503321, 296051, 88781, 453656, 246386, 39116, 403991, 196721, 561596, 354326, 147056, 511931, 304661, 97391, 462266, 254996, 47726, 412601]
[417564, 208719, 572019, 363174, 154329, 517629, 308784, 99939, 463239, 254394, 45549, 408849, 200004, 563304, 354459, 145614, 508914, 300069, 91224, 454524, 245679, 36834, 400134, 191289, 554589, 345744, 136899, 500199, 291354, 82509]
[247547, 155672, 63797, 544067, 452192, 360317, 268442, 176567, 84692, 564962, 473087, 381212, 289337, 197462, 105587, 13712, 493982, 402107, 310232, 218357, 126482, 34607, 514877, 423002, 331127, 239252, 147377, 55502, 535772, 443897]
[162181, 550996,

In [69]:
def calculateMinhashSignature(perm_mtx, shingle_array):
  sig_column = [np.inf] * len(perm_mtx)

  for i in range(len(shingle_array)):
    if(shingle_array[i] == 1):
      for j in range(len(sig_column)):
        if(perm_mtx[j][i] < sig_column[j]):
          sig_column[j] = perm_mtx[j][i]

  return sig_column

minhash_sig_rdd = shingles_int_rdd.map(lambda x: (x[0], calculateMinhashSignature(permutation_matrix, x[1])))
minhash_sig_rdd.take(10)

[('1578535646146199554',
  [1618, 22421, 24339, 9092, 9196, 17828, 11010, 2838, 27653, 14362]),
 ('1578535651984650240',
  [81103, 25781, 19299, 40067, 29251, 83, 4815, 75078, 10538, 18037]),
 ('1578535652248850432',
  [8023, 112091, 22134, 117242, 105901, 4073, 97530, 108, 10748, 8062]),
 ('1578535654149038080',
  [38998, 75551, 158529, 25682, 96241, 188663, 101625, 36543, 5918, 290617]),
 ('1578535658691629056',
  [1933, 74081, 4389, 4472, 18016, 21608, 28545, 17223, 58208, 28012]),
 ('1578535679713497089',
  [19993, 4151, 4704, 6992, 37966, 6593, 7440, 20793, 668, 18247]),
 ('1578535680585916417',
  [29443, 22421, 11739, 1427, 9406, 7853, 3975, 12288, 21143, 7222]),
 ('1578535686818304000',
  [1618, 5411, 48909, 14867, 166, 32843, 16680, 2838, 16733, 32212]),
 ('1578535704694779907',
  [61993, 495446, 375039, 24317, 16021, 25493, 81045, 25833, 111443, 164722]),
 ('1578535714945658882',
  [24508, 26306, 5334, 25052, 15916, 2813, 5445, 11553, 143, 12787])]

# DEBUG

In [67]:
def generatePermutation(arrayToHash, a, b):
  arrayToReturn = arrayToHash.copy()
  
  for i in range(len(arrayToHash)):
    arrayToReturn[i] = (a * arrayToHash[i] + b) % len(arrayToHash)

  print(arrayToHash, a, b, arrayToReturn)

  return arrayToReturn
  
def calculateMinhashSignature(perm_mtx, shingle_array):
  sig_column = [np.inf] * len(perm_mtx)

  for i in range(len(shingle_array)):
    if(shingle_array[i] == 1):
      for j in range(len(sig_column)):
        if(perm_mtx[j][i] < sig_column[j]):
          #print(perm_mtx[j][i])
          sig_column[j] = perm_mtx[j][i]

  return sig_column

n = int(input("Insert number n of permutation desired: "))
permutation_matrix = list()
tmp = list(range(0, 10))
array = [0,0,0,0,1,0,0,1,1,0]

print(tmp)

for i in range(n):
  #Generate 2 random numbers in order to have different hashing by randomly chaging only these two numbers
  a = np.random.randint(10) 
  b = np.random.randint(10)

  permutation_matrix.append(generatePermutation(tmp, a, b))
  
[print(*line) for line in permutation_matrix]
  
a = calculateMinhashSignature(permutation_matrix, array)
print(a)


Insert number n of permutation desired: 3
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9] 7 8 [8, 5, 2, 9, 6, 3, 0, 7, 4, 1]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9] 4 5 [5, 9, 3, 7, 1, 5, 9, 3, 7, 1]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9] 1 0 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
8 5 2 9 6 3 0 7 4 1
5 9 3 7 1 5 9 3 7 1
0 1 2 3 4 5 6 7 8 9
[4, 1, 4]
